<a href="https://colab.research.google.com/github/laysabelici/bootcamp-data-science-dio/blob/main/api_generativa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
sdw_api_url = 'https://sdw-2023-prd.up.railway.app'

**EXTRACT**

In [28]:
#EXTRAIR OS IDS DO ARQUIVO CSV

In [29]:
import pandas as pd
df = pd.read_csv('bootcamp-santander.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[1890, 1891]


In [30]:
#OBTER DADOS DE CADA ID USANDO API
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 1890,
    "name": "Gabriela",
    "account": {
      "id": 1980,
      "number": "0009-1",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 1823,
      "number": "**** **** **** 1590",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 1891,
    "name": "Joao",
    "account": {
      "id": 1981,
      "number": "0009-2",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 1824,
      "number": "**** **** **** 1591",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  }
]


**TRANSFORMAÇÃO**

In [31]:
!pip install openai

In [41]:
import requests

# Sua chave da API da OpenAI
api_key = ""

# URL da API da OpenAI
api_url = "https://api.openai.com/v1/engines/davinci/completions" # Substitua "some_endpoint" pelo endpoint real que deseja acessar

# Cabeçalho de autenticação
headers = {
    "Authorization": f"Bearer {api_key}"
}

# Realize uma solicitação GET (ou POST, etc.) para a API da OpenAI
response = requests.get(api_url, headers=headers)

# Verifique a resposta
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Erro na solicitação: {response.status_code}")
    print(response.text)


Erro na solicitação: 405
{
    "error": {
        "message": "Only POST requests are accepted.",
        "type": "invalid_request_error",
        "param": null,
        "code": "method_not_supported"
    }
}



In [42]:
open_api_key=''

In [43]:
import openai

openai.api_key = open_api_key

def generate_ai_news(user):
  completion = open.ChatCompletion.create(
      model='gpt-4',
      messages=[
            {
                'role':'system',
                'content':'Você é um especialista em marketing bancário.'
            },
            {
                'role':'user',
                'content':f'Crie uma mensagem para {user["name"]} sobre investimentos. (Até 100 caracteres)'
            }
          ]
  )

  return completion.choices[0].message.content.strip("\'")

  for user in users:
    news = generate_ai_news(user)
    print(news)
    user['news'].append({
        'icon':'',
        'description': news
    })

**CARREGAMENTO**

In [47]:
def update_user(user):
  response = requests.put(f'{sdw_api_url}/users/{user["id"]}', json=user)
  return True if response.status_code == 200 else False

for user in users:
  sucess = update_user(user)
  print(f'User {user["name"]} updated? {sucess}!')

User Gabriela updated? True!
User Joao updated? True!
